# STAC to OPeNDAP Demo - Using [`Catalog-To-Xpublish`](https://github.com/LimnoTech/Catalog-To-Xpublish)

**Author:** [Xavier R Nogueira](https://github.com/xaviernogueira)

**Overview:** This notebook demonstrates how to use the `Catalog-To-Xpublish` package to convert a STAC catalog of Zarr datasets into a [`xpublish`](https://xpublish.readthedocs.io/en/latest/) powered FastAPI server application. 

**Repo Info:**
* Built on a [`FastAPI`](https://fastapi.tiangolo.com/lo/) + [`xpublish`](https://github.com/xpublish-community/xpublish) stack.
* Fully compatible with [`xpublish` plugins](https://xpublish.readthedocs.io/en/latest/user-guide/plugins.html) as their [active community](https://github.com/xpublish-community) continues to develop them.
* Automated testing and pre-commit formatting protecting main branch functionality (46% coverage so far, potential next step?).
* Can be installed locally via `pip install git+https://github.com/LimnoTech/Catalog-To-Xpublish.git`

**Contents:**
1. [Setup](#Setup)
2. [Build application](#Build-application)
3. [Test default endpoints](#Test-default-endpoints)
4. [Use OPeNDAP endpoints](#Use-OPeNDAP-endpoints)
5. [Test OPeNDAP clients](#Test-OPeNDAP-clients)


# Setup

## Imports

In [98]:
# do imports
import sys
import pystac
import time
import json
from IPython.display import HTML
import hvplot.xarray
import geoviews.feature as gf
import subprocess
import requests
import xarray as xr
import netCDF4 as nc
from pathlib import Path
import cartopy.crs as ccrs

## Input STAC Catalog

**Notes on STAC Read-In:**
* `Catalog-To-Xpublish` can serve Zarr/NetCDF data linked as an:
    * "Asset" within a STAC "Collection".
    * "Asset" within a STAC "Item" that is nested within a STAC "Catalog". Note: At the moment, an Item is expected to have only one asset that corresponds to the dataset.
* STAC Collections/Catalogs are mapped to the server whether they contain datasets or not.
* If a bad link is included within a STAC, the link is skipped over, a warning message is logged, and the build proceeds without it.


**[Demo Catalog](https://code.usgs.gov/wma/nhgf/stac/-/tree/main/xpublish_sample_stac/catalog):**
* Contains both storage patterns mentioned above (i.e., Item vs Collection level).
* Contains sub-catalogs.
* Contains s3 hosted Zarr data.
* Put together by Amelia Snyder at my request...thank you!

In [2]:
catalog = pystac.Catalog.from_file(r'https://code.usgs.gov/wma/nhgf/stac/-/raw/main/xpublish_sample_stac/catalog/catalog.json')
catalog

id: nhgf-stac-catalog
description: Sample NHGF STAC catalog.
type: Catalog
id: conus404-daily
description: CONUS404 40 years of daily values for subset of model output variables derived from hourly values on cloud storage
type: Collection
https://stac-extensions.github.io/eo/v1.0.0/schema.json
https://stac-extensions.github.io/xarray-assets/v1.0.0/schema.json
https://stac-extensions.github.io/datacube/v2.2.0/schema.json
rel: root
href: https://code.usgs.gov/wma/nhgf/stac/-/raw/main/xpublish_sample_stac/catalog/catalog.json


In [3]:
for child in catalog.get_children():
    print(child)

<Collection id=conus404-daily>
<Catalog id=nhgf-stac-catalog>


### Collection example

In [4]:
catalog.get_child('conus404-daily')

id: conus404-daily
description: CONUS404 40 years of daily values for subset of model output variables derived from hourly values on cloud storage
type: Collection
https://stac-extensions.github.io/eo/v1.0.0/schema.json
https://stac-extensions.github.io/xarray-assets/v1.0.0/schema.json
https://stac-extensions.github.io/datacube/v2.2.0/schema.json
rel: root
href: https://code.usgs.gov/wma/nhgf/stac/-/raw/main/xpublish_sample_stac/catalog/catalog.json
type: application/json
rel: self
href: https://code.usgs.gov/wma/nhgf/stac/-/raw/main/xpublish_sample_stac/catalog/conus404-daily/collection.json


### Catalog w/ Items example

In [5]:
catalog.get_child('nhgf-stac-catalog')

id: nhgf-stac-catalog
description: Sample NHGF STAC catalog.
type: Catalog
id: GMO
"bbox: [-124.6875, 25.1875, -67.0625, 52.8125]"
start_datetime: 1950-01-01T00:00:00Z
end_datetime: 1999-12-31T00:00:00Z
"cube:dimensions: {'time': {'type': 'temporal', 'description': 'Time axis', 'extent': ['1950-01-01T00:00:00Z', '1999-12-31T00:00:00Z'], 'step': 'P1DT0H0M0S'}, 'x': {'type': 'spatial', 'axis': 'x', 'description': 'Longitude', 'extent': [-124.6875, -67.0625], 'step': 0.125}, 'y': {'type': 'spatial', 'axis': 'y', 'description': 'Latitude', 'extent': [-124.6875, -67.0625], 'step': 0.125}}"
datetime: None
"cube:variables: {'Prcp': {'dimensions': ['time', 'latitude', 'longitude'], 'type': 'data', 'description': 'daily_avg_precipitation', 'unit': 'mm/d'}, 'Tavg': {'dimensions': ['time', 'latitude', 'longitude'], 'type': 'data', 'description': 'daily_avg_air_temp', 'unit': 'C'}, 'Tmax': {'dimensions': ['time', 'latitude', 'longitude'], 'type': 'data', 'description': 'daily_max_air_temp', 'unit': 'C'}, 'Tmin': {'dimensions': ['time', 'latitude', 'longitude'], 'type': 'data', 'description': 'daily_min_air_temp', 'unit': 'C'}, 'Wind': {'dimensions': ['time', 'latitude', 'longitude'], 'type': 'data', 'description': 'daily_avg_wind_speed', 'unit': 'm/s'}}"
https://stac-extensions.github.io/datacube/v2.0.0/schema.json


# Build application

In [6]:
# import inputs from run_server.py and spin up server
sys.path.append(str(Path.cwd().parent))
from examples.run_server import (
    CATALOG_TYPE,
    CATALOG_PATH,
    APP_NAME,
    XPUBLISH_PLUGINS,
    CONFIG_LOGGING_DICT,
    LOCAL_HOST,
    LOCAL_PORT,
)

[2023-06-07T12:51:27] INFO - Spinning up server from stac catalog at https://code.usgs.gov/wma/nhgf/stac/-/raw/main/xpublish_sample_stac/catalog/catalog.json.
[2023-06-07T12:51:30] INFO - Mounting a Xpublish server @ /conus404-daily to the main application.
[2023-06-07T12:51:30] INFO - Mounting a Xpublish server @ /nhgf-stac-catalog to the main application.
[2023-06-07T12:51:30] INFO - Returning successfully created server application!


Code being executed in this section is shown below:

```python
catalog_to_xpublish.create_app(
    catalog_path=CATALOG_PATH,
    catalog_type=CATALOG_TYPE,
    app_name=APP_NAME,
    xpublish_plugins=XPUBLISH_PLUGINS,
    config_logging_dict=CONFIG_LOGGING_DICT,
)
```

Where the inputs are as follows:

In [7]:
# display inputs
INPUTS_DICT = {
    'catalog_type': CATALOG_TYPE,
    'catalog_path': CATALOG_PATH,
    'app_name': APP_NAME,
    'xpublish_plugins': XPUBLISH_PLUGINS,
    'config_logging_dict': CONFIG_LOGGING_DICT,
    'local_host': LOCAL_HOST,
    'local_port': LOCAL_PORT,
}
display(INPUTS_DICT)

{'catalog_type': 'stac',
 'catalog_path': 'https://code.usgs.gov/wma/nhgf/stac/-/raw/main/xpublish_sample_stac/catalog/catalog.json',
 'app_name': 'Xpublish Server',
 'xpublish_plugins': [xpublish_opendap.plugin.OpenDapPlugin],
 'config_logging_dict': {'level': 'INFO', 'log_file': 'xpublish_server.log'},
 'local_host': '127.0.0.1',
 'local_port': 8000}

In [8]:
# run the server (wait 15 seconds for it to start)
server_process = subprocess.Popen(['python', 'run_server.py'])
time.sleep(15)
server_process

<Popen: returncode: None args: ['python', 'run_server.py']>

# Test default endpoints

In [13]:
# set up base url
base_url = f'http://{LOCAL_HOST}:{LOCAL_PORT}'

# test that server is running
try:
    assert str(requests.get(base_url).json()) == "{'detail': 'Not Found'}"
    print('Server is running!')
except AssertionError:
    print('Server is not running!')

Server is running!


## Root catalog level (no datasets)

One can use `/json` at any level of the catalog to see a JSON representation of the catalog (in this case a STAC).

In [ ]:
endpoint1 = base_url + '/json'
print(f'Endpoint: {endpoint1}\nResponse:')
json.loads(requests.get(endpoint1).json())

To see if there are any "sub-catalogs" nested within a certain level of the catalog, one can use the `/catalogs` endpoint.

In [ ]:
endpoint2 = base_url + '/catalogs'
print(f'Endpoint: {endpoint2}\nResponse:')
requests.get(endpoint2).json()

At any point one can see what the parent catalog is using `/parent_catalog` (in this case we are at the root).

In [ ]:
endpoint3 = base_url + '/parent_catalog'
print(f'Endpoint: {endpoint3}\nResponse:')
requests.get(endpoint3).json()

## Sub-catalog level (Collection as a datasets)

Next we navigate to the 'conus404-daily' sub-catalog and explore the endpoints there. To do so, one can simply append `/{SUB_CATALOG_NAME}` to the root URL.

If a sub-catalog has datasets, one can see what those datasets are using the `/datasets` endpoint.

In [ ]:
endpoint4 = base_url + '/conus404-daily' + '/datasets'
print(f'Endpoint: {endpoint4}\nResponse:')
requests.get(endpoint4).json()

Same as the previous examples at the root level, we can query the sub-catalog level using `/catalogs`, `/parent_catalog`, and `/json`.

In [11]:
endpoint5 = base_url + '/conus404-daily' + '/catalogs'
print(f'Endpoint: {endpoint5}\nResponse:')
requests.get(endpoint5).json()

Endpoint: http://127.0.0.1:8000/conus404-daily/catalogs
Response:


[]

In [12]:
endpoint6 = base_url + '/conus404-daily' + '/parent_catalog'
print(f'Endpoint: {endpoint6}\nResponse:')
requests.get(endpoint6).json()

Endpoint: http://127.0.0.1:8000/conus404-daily/parent_catalog
Response:


'/'

**Here we can see that the "zarr-s3" dataset has metadata included in the STAC Collection.**

In [ ]:
endpoint7 = base_url + '/conus404-daily' + '/json'
print(f'Endpoint: {endpoint7}\nResponse:')
json.loads(requests.get(endpoint7).json())

## Sub-catalog level (Catalog w/ Items as datasets)

Next we navigate to the 'nhgf-stac-catalog' sub-catalog and explore the endpoints there.

In [15]:
endpoint8 = base_url + '/nhgf-stac-catalog' + '/datasets'
print(f'Endpoint: {endpoint8}\nResponse:')
requests.get(endpoint8).json()

Endpoint: http://127.0.0.1:8000/nhgf-stac-catalog/datasets
Response:


['GMO', 'GMO_New', 'PRISM', 'PRISM_v2', 'UofIMETDATA']

In [16]:
endpoint9 = base_url + '/nhgf-stac-catalog' + '/catalogs'
print(f'Endpoint: {endpoint9}\nResponse:')
requests.get(endpoint9).json()

Endpoint: http://127.0.0.1:8000/nhgf-stac-catalog/catalogs
Response:


[]

In [17]:
endpoint10 = base_url + '/nhgf-stac-catalog' + '/parent_catalog'
print(f'Endpoint: {endpoint10}\nResponse:')
requests.get(endpoint10).json()

Endpoint: http://127.0.0.1:8000/nhgf-stac-catalog/parent_catalog
Response:


'/'

**Here we can see that the Catalog contains items that contain the datasets.**

In [18]:
endpoint11 = base_url + '/nhgf-stac-catalog' + '/json'
print(f'Endpoint: {endpoint11}\nResponse:')
json.loads(requests.get(endpoint11).json())

Endpoint: http://127.0.0.1:8000/nhgf-stac-catalog/json
Response:


{'type': 'Catalog',
 'id': 'nhgf-stac-catalog',
 'stac_version': '1.0.0',
 'description': 'Sample NHGF STAC catalog.',
 'links': [{'rel': 'root',
   'href': '../catalog.json',
   'type': 'application/json'},
  {'rel': 'item', 'href': './GMO/item.json', 'type': 'application/json'},
  {'rel': 'item', 'href': './GMO_New/item.json', 'type': 'application/json'},
  {'rel': 'item', 'href': './PRISM/item.json', 'type': 'application/json'},
  {'rel': 'item', 'href': './PRISM_v2/item.json', 'type': 'application/json'},
  {'rel': 'item',
   'href': './UofIMETDATA/item.json',
   'type': 'application/json'},
  {'rel': 'self',
   'href': 'https://code.usgs.gov/wma/nhgf/stac/-/raw/main/xpublish_sample_stac/catalog/GDP_catalog/catalog.json',
   'type': 'application/json'},
  {'rel': 'parent', 'href': '../catalog.json', 'type': 'application/json'}]}

## `Xpublish` endpoints

`Xpublish` includes some default endpoints that are available when datasets are present.
* `datasets/{dataset_id}` - returns the HTML representation of the dataset.
* `datasets/{dataset_id}/keys` - returns variable names of the dataset.
* `datasets/{dataset_id}/info` - returns dataset info.
* `datasets/{dataset_id}/dict` - returns the dataset as a dictionary (won't be tested, very slow).

**Note:** From here on out we will be testing with `nhgf-stac-catalog/UofIMETDATA.zarr`.

In [22]:
dataset_url = base_url + '/nhgf-stac-catalog' + '/datasets' + '/UofIMETDATA'
print(f'Endpoint: {dataset_url}\nResponse:')
display(HTML(requests.get(dataset_url).text))

Endpoint: http://127.0.0.1:8000/nhgf-stac-catalog/datasets/UofIMETDATA
Response:


In [23]:
endpoint12 = dataset_url + '/keys'
print(f'Endpoint: {endpoint12}\nResponse:')
display(requests.get(endpoint12).json())

Endpoint: http://127.0.0.1:8000/nhgf-stac-catalog/datasets/UofIMETDATA/keys
Response:


['crs',
 'day',
 'lat',
 'lon',
 'max_air_temperature',
 'max_relative_humidity',
 'min_air_temperature',
 'min_relative_humidity',
 'precipitation_amount',
 'specific_humidity',
 'surface_downwelling_shortwave_flux_in_air',
 'wind_speed']

In [ ]:
endpoint13 = dataset_url + '/info'
print(f'Endpoint: {endpoint13}\nResponse:')
display(requests.get(endpoint13).json())

# Test OPeNDAP endpoints

`Catalog-To-Xpublish` is extendable using any `xpublish` plugin. In this example we use the [`xpublish_opendap`](https://github.com/xpublish-community/xpublish-opendap).

The following endpoints are added to the application:
* `datasets/{dataset_id}/opendap.dds` - returns the OPeNDAP Dataset Descriptor Structure (DDS) response.
* `datasets/{dataset_id}/opendap.das` - returns the OPeNDAP Data Attribute Structure (DAS) response.
* `datasets/{dataset_id}/opendap.dods` - returns the OPeNDAP Data Object Structure (DODS) response.

Additionally one can use OPeNDAP "constraint expressions" to sub-select data on the server side.

## First we will test the `.dds` and `.das` endpoints. 

In this case we are getting `'precipitation_amount'` data from the `nhgf-stac-catalog/UofIMETDATA.zarr` dataset.

In [26]:
opendap_url = dataset_url + '/opendap'
variable = 'precipitation_amount'

# here we can see a shape summary of the ACETLSM
endpoint14 = opendap_url + '.dds?' + variable
print(f'Endpoint: {endpoint14}\nResponse:')
print(requests.get(endpoint14).text)

Endpoint: http://127.0.0.1:8000/nhgf-stac-catalog/datasets/UofIMETDATA/opendap.dds?precipitation_amount
Response:
Dataset {
    Grid {
      Array:
        Float32 precipitation_amount[day = 15861][lat = 585][lon = 1386];
      Maps:
        Float64 day[day = 15861];
        Float64 lat[lat = 585];
        Float64 lon[lon = 1386];
    } precipitation_amount;
} UofIMETDATA;



In [188]:
# here we can show an example of sub-setting the ACETLSM
time_range = '[0:1]'
y_range = '[100:300]'
x_range = '[100:300]'
subset_string = f'{time_range}{y_range}{x_range}'
endpoint15 = opendap_url + f'.dds?{variable}{subset_string}'
print(f'Endpoint: {endpoint15}\nResponse:')
print(requests.get(endpoint15).text)

Endpoint: http://127.0.0.1:8000/nhgf-stac-catalog/datasets/UofIMETDATA/opendap.dds?precipitation_amount[0:1][100:300][100:300]
Response:
Dataset {
    Grid {
      Array:
        Float32 precipitation_amount[day = 2][lat = 201][lon = 201];
      Maps:
        Float64 day[day = 2];
        Float64 lat[lat = 201];
        Float64 lon[lon = 201];
    } precipitation_amount;
} UofIMETDATA;



In [189]:
# here we can see units and full attribute metadata
endpoint16 = opendap_url + f'.das?{variable}'
print(f'Endpoint: {endpoint16}\nResponse:')
print(requests.get(endpoint16).text)

Endpoint: http://127.0.0.1:8000/nhgf-stac-catalog/datasets/UofIMETDATA/opendap.das?precipitation_amount
Response:
Attributes {
    precipitation_amount {
        String coordinate_system "WGS84,EPSG:4326";
        String description "Daily Accumulated Precipitation";
        String dimensions "lon lat time";
        String grid_mapping "crs";
        String long_name "pr";
        String standard_name "pr";
        String units "mm";
    }
    String Conventions "CF-1.0";
    String Metadata_Conventions "Unidata Dataset Discovery v1.0";
    String acknowledgement "Whenever you publish research based on data from this archive, please reference this data by using the phrase -daily gridded meteorological data(METDATA) for the continental US- and by citing the article (Abatzoglou,2012). Further, appropriately acknowledge the National Science Foundation (NSF), Idaho EPSCoR and the individual investigators responsible for the data set.";
    String author "John Abatzoglou - University of Ida

## Test OPeNDAP clients

With a variable and sub-selection of data choses, we can now use the `.dods` response (binary) to open the data in `xarray` and as a `NetCDF`.

In [231]:
dods_url = opendap_url + f'?{variable}{time_range}{y_range}{x_range}'
print(f'Endpoint with subselection: {dods_url}')

Endpoint with subselection: http://127.0.0.1:8000/nhgf-stac-catalog/datasets/UofIMETDATA/opendap?precipitation_amount[0:1][100:300][100:300]


### Open our subset of data w/ `netCDF4`

In [232]:
%%time
nc_dataset = nc.Dataset(dods_url)

CPU times: total: 141 ms
Wall time: 2.79 s


In [233]:
nc_dataset.data_model

'NETCDF3_CLASSIC'

In [234]:
nc_dataset.dimensions

{'day': <class 'netCDF4._netCDF4.Dimension'>: name = 'day', size = 2,
 'lat': <class 'netCDF4._netCDF4.Dimension'>: name = 'lat', size = 201,
 'lon': <class 'netCDF4._netCDF4.Dimension'>: name = 'lon', size = 201}

In [230]:
nc_dataset.variables

{'day': <class 'netCDF4._netCDF4.Variable'>
 float64 day(day)
 unlimited dimensions: 
 current shape = (201,)
 filling off,
 'lat': <class 'netCDF4._netCDF4.Variable'>
 float64 lat(lat)
 unlimited dimensions: 
 current shape = (201,)
 filling off,
 'lon': <class 'netCDF4._netCDF4.Variable'>
 float64 lon(lon)
 unlimited dimensions: 
 current shape = (201,)
 filling off,
 'precipitation_amount': <class 'netCDF4._netCDF4.Variable'>
 float32 precipitation_amount(day, lat_1, lon_1)
 unlimited dimensions: 
 current shape = (201, 585, 1386)
 filling off}

### Open our subset of data w/ `xarray`

**NOTES:** 
* For `xarray`, although the `.dods` endpoint exists, one should pass in the opendap endpoint without `.dods`.
* Not all `engine` params are supported. See this [issue](https://github.com/xpublish-community/xpublish-opendap/issues/18).
* When one accesses a single variable, information about the dimensions is lost!

#### Get Zarr chunking information from `datasets/{DATASET_ID}/zarr/.zmetadata`

In [210]:
zarr_url = dataset_url + '/zarr/.zmetadata'
zarr_url

'http://127.0.0.1:8000/nhgf-stac-catalog/datasets/UofIMETDATA/zarr/.zmetadata'

In [211]:
zarr_dict = dict(requests.get(zarr_url).json())
chunks = zarr_dict['metadata'][f'precipitation_amount/.zarray']['chunks']

In [212]:
# get chunks for Zarr endpoint for more efficient read-in
dims_dict = dict(requests.get(dataset_url + '/info').json())['dimensions']
del dims_dict['crs']
chunks_dict = {}
for i, c in enumerate(chunks):
    chunks_dict[list(dims_dict.keys())[i]] = c
print(f'Zarr chunking scheme: {chunks_dict}')

Zarr chunking scheme: {'day': 2190, 'lat': 150, 'lon': 150}


#### Use `xarray.open_dataset`

In [254]:
%%time
# get the chunked data in xarray
xr_dataset = xr.open_dataset(
    opendap_url,
    engine='netcdf4',
    chunks=chunks_dict,
)[variable].copy(deep=True)
xr_dataset

CPU times: total: 844 ms
Wall time: 6.8 s


<xarray.DataArray 'precipitation_amount' (day: 15861, lat: 585, lon: 1386)>
dask.array<open_dataset-precipitation_amount, shape=(15861, 585, 1386), dtype=float32, chunksize=(2190, 150, 150), chunktype=numpy.ndarray>
Coordinates:
  * day      (day) float64 2.885e+04 2.886e+04 2.886e+04 ... 4.471e+04 4.471e+04
  * lat      (lat) float64 49.4 49.36 49.32 49.28 ... 25.19 25.15 25.11 25.07
  * lon      (lon) float64 -124.8 -124.7 -124.7 -124.6 ... -67.14 -67.1 -67.06

#### And now for fun, let's visualize our data!

In [70]:
import dask.distributed

In [ ]:
dask.distributed.Client()

In [279]:
# keep only the first dat
east_coast = xr_dataset[0:1,:,1000:].compute().squeeze()
east_coast

<xarray.DataArray 'precipitation_amount' (lat: 585, lon: 386)>
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)
Coordinates:
    day      float64 2.885e+04
  * lat      (lat) float64 49.4 49.36 49.32 49.28 ... 25.19 25.15 25.11 25.07
  * lon      (lon) float64 -83.1 -83.06 -83.02 -82.97 ... -67.14 -67.1 -67.06

In [280]:
plot = east_coast.where(east_coast != 0).hvplot.image(
    cmap='GnBu',
    width=800,
    height=400,
    crs='4326',
    tiles='StamenTerrainRetina',
) * gf.coastline()
display(plot)

:Overlay
   .Tiles.I     :Tiles   [x,y]
   .Image.I     :Image   [lon,lat]   (precipitation_amount)
   .Coastline.I :Feature   [Longitude,Latitude]